In [7]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

In [8]:
api_key = "78d406f975874c20ac2298c6d83e2f7d"
azure_endpoint = "https://nitoropenai.openai.azure.com/"
api_version = "2022-12-01"

llm = AzureOpenAI(
    model="text-davinci-003",
    deployment_name="text-davinci-003",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [9]:
# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [10]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [17]:
PERSIST_DIR = "./bot-storage"

data = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(data)
index.storage_context.persist(persist_dir=PERSIST_DIR)

memory = ChatMemoryBuffer.from_defaults()

In [18]:
chat_engine = index.as_chat_engine(
    chat_mode="context", # different chat_mode includes - ReAct, Best, context, Simple, Condensed
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        "about an essay discussing Paul Grahams life."
    ),
)

In [19]:
response = chat_engine.chat("Hello!")
response

AgentChatResponse(response=' Hi there! How can I help you?', sources=[ToolOutput(content='system: You are a chatbot, able to have normal interactions, as well as talkabout an essay discussing Paul Grahams life.\nContext information is below.\n--------------------\nfile_path: data/paul_graham_essay.txt\n\nNor had I changed my grad student lifestyle significantly since we started. So when Yahoo bought us it felt like going from rags to riches. Since we were going to California, I bought a car, a yellow 1998 VW GTI. I remember thinking that its leather seats alone were by far the most luxurious thing I owned.\n\nThe next year, from the summer of 1998 to the summer of 1999, must have been the least productive of my life. I didn\'t realize it at the time, but I was worn out from the effort and stress of running Viaweb. For a while after I got to California I tried to continue my usual m.o. of programming till 3 in the morning, but fatigue combined with Yahoo\'s prematurely aged culture and 

In [20]:
response.response

' Hi there! How can I help you?'

In [21]:
response = chat_engine.chat("What did Paul Graham do growing up?")
response.response

' Paul Graham grew up in Pittsburgh, Pennsylvania. He attended a private school and was an avid reader. He was also interested in computers and programming from a young age, and he wrote his first computer program when he was 12. He went on to attend Harvard University, where he studied computer science and philosophy.'

In [22]:
response = chat_engine.chat("Can you tell me more?")
response.response

' Sure! After graduating from Harvard, Paul Graham worked as a software developer and consultant for several years. He then co-founded Viaweb, an online store-building software company, which was acquired by Yahoo in 1998. After that, he founded Y Combinator, a startup accelerator, in 2005. He has also written several essays and books on topics such as technology, startups, and entrepreneurship.'

In [23]:
chat_engine.reset()

In [25]:
# Streaming support
response = chat_engine.stream_chat("What did Paul Graham do after YC?")
for token in response.response_gen:
    print(token, end="")

 After YC, Paul Graham wrote lots of essays about different topics, worked on spam filters, did some painting, had dinners for a group of friends every Thursday night, bought a building in Cambridge, and started dating Jessica Livingston. He also gave a talk at the Harvard Computer Society about how to start a startup, and eventually he and Jessica, along with Robert and Trevor, started Y Combinator. He also worked on angel investing, and eventually he and Jessica, along with Robert and Trevor, started Y Combinator.

In [26]:
response = chat_engine.chat("Who is Sundat Pichai?")
response.response

' Sundar Pichai is the CEO of Google. He was previously the Product Chief at Google, and before that he was the Vice President of Product Management and Innovation at Google. He has also held various positions at McKinsey & Company and Applied Materials.'

## Chat Engine - Condense Question Mode

https://docs.llamaindex.ai/en/stable/examples/chat_engine/chat_engine_condense_question/

In [32]:
chat_engine.reset()

In [33]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

In [34]:
response = chat_engine.chat("What did Paul Graham do after YC?")
response

Querying with: What was the next step in Paul Graham's career after he founded Y Combinator?


AgentChatResponse(response=' After founding Y Combinator, Paul Graham continued to write essays about various topics, worked on spam filters, painted, cooked for groups, and bought another building in Cambridge. He also went on to write a book called Hackers & Painters, which was a collection of his essays.', sources=[ToolOutput(content=' After founding Y Combinator, Paul Graham continued to write essays about various topics, worked on spam filters, painted, cooked for groups, and bought another building in Cambridge. He also went on to write a book called Hackers & Painters, which was a collection of his essays.', tool_name='query_engine', raw_input={'query': "What was the next step in Paul Graham's career after he founded Y Combinator?"}, raw_output=Response(response=' After founding Y Combinator, Paul Graham continued to write essays about various topics, worked on spam filters, painted, cooked for groups, and bought another building in Cambridge. He also went on to write a book cal

In [35]:
response = chat_engine.chat("What about after that?")
response

Querying with: What other activities did Paul Graham pursue after founding Y Combinator and writing his book Hackers & Painters?


AgentChatResponse(response=' Paul Graham pursued spam filtering, painting, cooking for groups, and investing in startups. He also gave talks to the Harvard Computer Society about how to start a startup.', sources=[ToolOutput(content=' Paul Graham pursued spam filtering, painting, cooking for groups, and investing in startups. He also gave talks to the Harvard Computer Society about how to start a startup.', tool_name='query_engine', raw_input={'query': 'What other activities did Paul Graham pursue after founding Y Combinator and writing his book Hackers & Painters?'}, raw_output=Response(response=' Paul Graham pursued spam filtering, painting, cooking for groups, and investing in startups. He also gave talks to the Harvard Computer Society about how to start a startup.', source_nodes=[NodeWithScore(node=TextNode(id_='9c406776-94d7-48db-9d17-31cebc3bc0d4', embedding=None, metadata={'file_path': 'data/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain'